In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import ee, geemap
import shapely
import seaborn as sns

In [19]:
ee.Initialize()

In [33]:
santiago = gpd.read_file('Datos/C2017-2012_RM_pob_socioec_DIST.gpkg').to_crs(32619).reset_index()
santiago.head()

manzanas = gpd.read_file("Datos/R13/COMUNA_C17.shp").to_crs(32619)

comunas = manzanas[manzanas.intersects(santiago.geometry.unary_union)]
comunas = comunas.drop(40)
comunas = comunas.reset_index().drop(columns='index').to_crs(32619)
aoi = geemap.geopandas_to_ee(comunas)
comunas.explore()

Ahora se generará un mapa de temperatura superficial para Santiago usando la definición de LST. El LST (land surface temperature) se define como
$$LST = \frac{BT}{1 + \frac{0.00115*BT}{1.4388}} \cdot \ln (\varepsilon)$$
con
$$
BT = \frac{1321.08}{\ln \Big(\frac{777.89}{L} \Big) + 1} - 273.15 \qquad 
\varepsilon = 0.004\cdot \frac{NDVI - NDVImin}{NDVImax - NDVImin} + 0.986 \qquad
NDVI = \frac{NIR - RED}{NIR + RED}
$$

In [34]:
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )


def fmask(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    return image.updateMask(qaMask)


def add_Layers(image):

    expr_NDVI = 'NDVI = (NIR - Red) / (NIR + Red)'
    ndvi = ee.Image().expression(
        expression=expr_NDVI, opt_map={'NIR': image.select('SR_B5'), 'Red': image.select('SR_B4')}
    ).rename('NDVI')


    stats = ndvi.reduceRegion(reducer=ee.Reducer.minMax(), geometry=aoi)
    Pv = ndvi.unitScale(stats.get('NDVI_min'), stats.get('NDVI_max'))

    expr_epsilon = 'e = 0.004*pow(Pv, 2) + 0.986'
    epsilon = ee.Image().expression(
        expression=expr_epsilon, opt_map={'Pv': Pv}
    ).rename('e')

    thermal = image.select('ST_B10').rename('thermal')
    lst_yogyakarta = thermal.expression(
    '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
        'TB': thermal.select('thermal'),
        'em': epsilon 
    }).rename('LST_Yogyakarta')

    expr_BT = 'BT = 1321.08 / (log(774.8853/L)+1) - 273.15'
    bt = ee.Image().expression(
        expression=expr_BT, opt_map={'L': image.select(['ST_B10'])}
    ).rename('BT')

    expr_LST = 'LST = BT / (1 + 0.00115*BT/1.4388) + log(e)'
    lst = ee.Image().expression(
        expression=expr_LST, opt_map={'BT': bt, 'e': epsilon}
    ).rename('LST')

    lst_celsius = lst.add(-273).rename('LST_Celsius')
    st_b10_celsius = image.select(['ST_B10']).add(-273).rename('ST_B10_Celsius')

    return image.addBands(lst_celsius, None, True)\
                .addBands(lst_yogyakarta, None, True)\
                .addBands(st_b10_celsius, None, True)\
                .addBands(ndvi, None, True)

In [38]:
fechas = ["2023-01-01","2023-04-01"]

collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(ee.Filter.date(fechas[0], fechas[1]))
# collection = collection.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))
collection = collection.map(apply_scale_factors)
collection = collection.map(add_Layers)
collection = collection.map(fmask)
collection = collection.sort('system:time_start', False)

images = collection.aggregate_array('system:id').getInfo()
print(f'Se obtuvieron {len(images)} imágenes')

Se obtuvieron 32688 imágenes


In [41]:
image1 = collection.first().clip(aoi).reproject(crs='EPSG:32619', scale=30)
image1

In [26]:
images[0]

'LANDSAT/LC08/C02/T1_L2/LC08_233084_20230323'

In [95]:
reducer_geom = ee.Reducer.geometricMedian(1)
geomediana = collection.reduce(reducer_geom)
geomediana

In [42]:
import geemap
# image = collection.first()
visualization = {
    'min': 0,'max': 1, 
    "bands": ["ST_B10_Celsius"]
}
Map = geemap.Map(center=[-33.473, -70.653], zoom=10)
Map.add_layer(image1, visualization)
Map.add_layer(aoi)
Map

EEException: Collection.first: Error in map(ID=LC08_088082_20230331):
Image.reduceRegion: Too many pixels in the region. Found 2403049879, but maxPixels allows only 10000000.
Ensure that you are not aggregating at a higher resolution than you intended; that is a frequent cause of this error. If not, then you may set the 'maxPixels' argument to a limit suitable for your computation; set 'bestEffort' to true to aggregate at whatever scale results in 'maxPixels' total pixels; or both.

# Perspectivas Socioeconómicas

In [41]:
df = pd.read_html("Datos/reportesEstadisticos-unidadTerritorial.xls") # Datos extraidos de: https://cead.spd.gov.cl/estadisticas-delictuales/#descargarExcel
df = df[1]
df.columns = ["Comuna","2022","2023"] # Hay que limpiar estos datos para poder hacer el join entre comunas (del censo, georeferenciadas) y la tasa de crimenes sociales 
# comunas = gpd.read_file("Datos/R13/COMUNA_C17.shp")
# comunas = comunas.to_crs("EPSG:4326")
# comunas.join(df, on="NOM_CUMUNA")
df.drop([0, 1, 2, 35, 39, 43, 48, 54],axis=0)

,Comuna,2022,2023
3,Santiago,"5.234,7755467597","5.557,6592692631"
4,Cerrillos,"3.949,5648690156","4.093,9373750833"
5,Cerro Navia,"1.868,4588041581","2.322,5044636188"
6,ConchalÃ­,"2.294,4647210721","2.570,4905649414"
7,El Bosque,"1.980,1286207440","2.048,2478772810"
8,EstaciÃ³n Central,"3.674,9301675978","4.784,0579907866"
9,Huechuraba,"2.268,2939460374","2.470,0950299263"
10,Independencia,"2.460,7860122340","2.417,5390084930"
11,La Cisterna,"4.063,6443496886","4.219,8920859761"
12,La Florida,"2.747,9574328765","2.683,4088830765"


In [55]:
#https://www.desarrollosocialyfamilia.gob.cl/storage/docs/INDICE-DE-PRIORIDAD-SOCIAL-2022_V2.pdf quizá extraer esos datos